In [ ]:
# !pip install PyMuPDF

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdfplumber
from pdf2docx import Converter
from pdfminer3.layout import LAParams, LTTextBox
from pdfminer3.pdfpage import PDFPage
from pdfminer3.pdfinterp import PDFResourceManager
from pdfminer3.pdfinterp import PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.converter import TextConverter
import io
import docx2txt
from docx import Document
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re
import os
import glob
import win32com.client
import PyPDF2
#import camelot
import fitz
import sys
from pdfrw import PdfReader
import operator
import wx
from fuzzywuzzy import process

C:\Users\VEERANI2\.conda\envs\ana_envi\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



<h2> using pdfplumber </h2>

In [ ]:
# with pdfplumber.open(r'noccodes.pdf') as pdf:
#     first_page = pdf.pages[487]
#     print(first_page.extract_text())

<h2> using pdfminer </h2>

In [ ]:
file_object = open('resume_text.txt', 'a')
resource_manager = PDFResourceManager()
fake_file_handle = io.StringIO()
converter = TextConverter(resource_manager, fake_file_handle, laparams=LAParams())
page_interpreter = PDFPageInterpreter(resource_manager, converter)

with open('resume.pdf', 'rb') as fh:

    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
        page_interpreter.process_page(page)
#         print()
        file_object.write(fake_file_handle.getvalue())
#     text = fake_file_handle.getvalue()
#         print(text)

# close open handles
converter.close()
fake_file_handle.close()
file_object.close()
print("Done...")

<h2> using pdf to word </h2>

In [ ]:
word = win32com.client.Dispatch("Word.Application")
word.visible = 0

In [ ]:
def Converter_pdf2docx(source,destination):
    pdf_file  = source
    docx_file = destination
    cv = Converter(pdf_file)
    cv.convert(docx_file, start=0, end=None)
    cv.close()

In [ ]:
Converter_pdf2docx('resume.pdf','text.docx')

In [2]:
noc_codes=pd.read_csv("noc_names.csv", encoding= 'unicode_escape',dtype = {'NOC Code': str,'Code Division': str,
                                                      'Code Name': str})

In [3]:
noc_codes.tail()

,NOC Code,Code Division,Code Name
496,9616,D,Labourers in textile processing
497,9617,D,Labourers in food and beverage processing
498,9618,D,Labourers in fish and seafood processing
499,9619,D,"Other labourers in processing, manufacturing a..."
500,NaN,NaN,NaN


In [ ]:
string1 = '9616'
string2 = '9617'
  
# opening a text file
file1 = open("nocedited_text.txt", "r")

def findword(string1,file1):
    # setting flag and index to 0
    flag = 0
    index = 0
    # Loop through the file line by line
    for line in file1:  
        index+=1 

        # checking string is present in line or not
        if string1 in line:
            flag = 1
            break 

    # checking condition for string found or not
    if flag == 0:
        print('String', string1 , 'Not Found') 
    else:
        print('String', string1, 'Found In Line', index)

    # closing text file    
     
    return index


In [ ]:
index1=findword(string1,file1)
index2=findword(string2,file1)
file1.close()

In [ ]:
file_contents = file1.read()

In [ ]:
print(file_contents)

In [ ]:
# os.path.abspath(os.getcwd())

In [ ]:
MY_TEXT = docx2txt.process("resume.docx")
with open("resume_text.txt", "w") as text_file:
    print(MY_TEXT, file=text_file)

In [ ]:
with open('noc_text.txt') as f:
    contents = f.read()

In [ ]:
doc = Document()
 
files = glob.glob("*.txt")
print(files)
file = input("Scrivi il nome del file senza .txt: ") + ".txt"


with open(file, 'rb', encoding= 'unicode_escape') as openfile:
#     openfile.encode('utf-8').strip()
    line = openfile.read()
    doc.add_paragraph(line)
    doc.save(file + ".docx")

os.system(file + ".docx")

In [ ]:
def check_header(filename):
    with open(filename) as f:
        first = f.read(1)
    return first not in '.-0123456789'

In [ ]:
check_header(file_contents)

In [4]:
#Extracting headings and paragraphs
def headers_para(doc, size_tag):
    """Scrapes headers & paragraphs from PDF and return texts with element tags.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param size_tag: textual element tags for each size
    :type size_tag: dict
    :rtype: list
    :return: texts with pre-prended element tags
    """
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                # REMEMBER: multiple fonts and sizes are possible IN one block

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = size_tag[s['size']] + s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = size_tag[s['size']] + s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = size_tag[s['size']] + s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += " " + s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = size_tag[s['size']] + s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
                    block_string += "|"

                header_para.append(block_string)

    return header_para

def font_tags(font_counts, styles):
    """Returns dictionary with font sizes as keys and tags as value.
    :param font_counts: (font_size, count) for all fonts occuring in document
    :type font_counts: list
    :param styles: all styles found in the document
    :type styles: dict
    :rtype: dict
    :return: all element tags based on font-sizes
    """
    p_style = styles[font_counts[0][0]]  # get style for most used font by count (paragraph)
    p_size = p_style['size']  # get the paragraph's size

    # sorting the font sizes high to low, so that we can append the right integer to each tag 
    font_sizes = []
    for (font_size, count) in font_counts:
        font_sizes.append(float(font_size))
    font_sizes.sort(reverse=True)

    # aggregating the tags for each font size
    idx = 0
    size_tag = {}
    for size in font_sizes:
        idx += 1
        if size == p_size:
            idx = 0
            size_tag[size] = '<p>'
        if size > p_size:
            size_tag[size] = '<h{0}>'.format(idx)
        elif size < p_size:
            size_tag[size] = '<s{0}>'.format(idx)

    return size_tag

def fonts(doc, granularity=False):
    """Extracts fonts and their usage in PDF documents.
    :param doc: PDF document to iterate through
    :type doc: <class 'fitz.fitz.Document'>
    :param granularity: also use 'font', 'flags' and 'color' to discriminate text
    :type granularity: bool
    :rtype: [(font_size, count), (font_size, count}], dict
    :return: most used fonts sorted by count, font style information
    """
    styles = {}
    font_counts = {}

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # block contains text
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if granularity:
                            identifier = "{0}_{1}_{2}_{3}".format(s['size'], s['flags'], s['font'], s['color'])
                            styles[identifier] = {'size': s['size'], 'flags': s['flags'], 'font': s['font'],
                                                  'color': s['color']}
                        else:
                            identifier = "{0}".format(s['size'])
                            styles[identifier] = {'size': s['size'], 'font': s['font']}

                        font_counts[identifier] = font_counts.get(identifier, 0) + 1  # count the fonts usage

    font_counts = sorted(font_counts.items(), key=operator.itemgetter(1), reverse=True)

    if len(font_counts) < 1:
        raise ValueError("Zero discriminating fonts found!")

    return font_counts, styles

def headings(doc):
    font_counts_, styles_=fonts(doc)
    size_tag_=font_tags(font_counts_, styles_)
    header_para_=headers_para(doc,size_tag_)
    headings=list()
    for val in header_para_:
        if "<h" in val:
            headings.append(val)
    val1=[sub.split("|") for sub in headings]
    flat_list = [item for sublist in val1 for item in sublist]
    df = pd.DataFrame(flat_list)
    return df

def sent_tags(doc):
    header_para = []  # list with headers and paragraphs
    first = True  # boolean operator for first header
    previous_s = {}  # previous span

    for page in doc:
        blocks = page.getText("dict")["blocks"]
        for b in blocks:  # iterate through the text blocks
            if b['type'] == 0:  # this block contains text

                # REMEMBER: multiple fonts and sizes are possible IN one block

                block_string = ""  # text found in block
                for l in b["lines"]:  # iterate through the text lines
                    for s in l["spans"]:  # iterate through the text spans
                        if s['text'].strip():  # removing whitespaces:
                            if first:
                                previous_s = s
                                first = False
                                block_string = s['text']
                            else:
                                if s['size'] == previous_s['size']:

                                    if block_string and all((c == "|") for c in block_string):
                                        # block_string only contains pipes
                                        block_string = s['text']
                                    if block_string == "":
                                        # new block has started, so append size tag
                                        block_string = s['text']
                                    else:  # in the same block, so concatenate strings
                                        block_string += s['text']

                                else:
                                    header_para.append(block_string)
                                    block_string = s['text']

                                previous_s = s

                    # new block started, indicating with a pipe
    #                 block_string = list(map(lambda st: str.replace(st, ",", " and "), block_string)) 
                    block_string += "|"

                header_para.append(block_string)

    return header_para

In [5]:
doc = fitz.open('noccodes_edited.pdf')

In [ ]:
doc

In [ ]:

a_string = doc.name

split_string = a_string.split(".", 1)

substring = split_string[0]
# substring=substring.replace("\\","").replace(" ","")

header_para = []  # list with headers and paragraphs
first = True  # boolean operator for first header
previous_s = {}  # previous span
header_para=sent_tags(doc)

In [ ]:
headings(doc).to_csv("headings.csv")

In [6]:
font_counts_, styles_=fonts(doc)
size_tag_=font_tags(font_counts_, styles_)
header_para_=headers_para(doc,size_tag_)

In [ ]:
header_para_

In [ ]:
for i in range(0, len(header_para_)):
    print (header_para_[1])

In [ ]:
test_list[5 : 8] = [''.join(test_list[5 : 8])]

In [ ]:
for i in range(0, len(header_para_)):
    if (header_para_[i].tolower().contains('<h3>')):
        

In [ ]:
md = [idx for idx, s in enumerate(header_para_) if "<h3>" in s]

In [ ]:
header_para_[md[0]]
header_para_[md[1]]

In [ ]:
len(header_para_) - 1

In [ ]:
md.append(len(header_para_) - 1)

In [ ]:
md

In [ ]:
d={}
for i in range(0,len(md)):
#     print(i)
    d["string{0}".format(i)] = ''.join(header_para_[md[i-1]:md[i]])

In [ ]:
keys_=d.keys()
# keys_

In [ ]:
d1={}
for val in keys_:
#     print(val)
    d1[d[val].split('|', 1)[0]]=d[val]
#     del d[val] 

In [ ]:
d1.keys()

In [ ]:
d1 = {x.replace('<h3>', ''): v 
     for x, v in d1.items()}
d1 = {x.replace('\xa0', ''): v 
     for x, v in d1.items()}

In [ ]:
d1['9534 Furniture finishers and refinishers']

In [ ]:
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return " "
#         return "The strings are {} edits away".format(distance[row][col])

In [ ]:
with open('resume_text.txt') as f:
    contents = f.read()

In [ ]:
for val in d1.keys():
    Distance = levenshtein_ratio_and_distance(contents,d1[val])
#     print(Distance)
    Ratio = levenshtein_ratio_and_distance(contents,d1[val],ratio_calc = True)
    print("% match between resume and "+val+": "+ Ratio)

In [ ]:
list_=[]
for val in d1.keys():
    list_.append(val+" "+d1[val])

In [ ]:
list1=list_[10:20]

In [ ]:
Ratios = process.extract(contents,list1)
print(Ratios)

In [ ]:
highest = process.extractOne(contents,list_)
print(highest)

In [ ]:
def cos_dist(text1, text2, q=3, pad=True):
    from scipy import spatial
    text1, text2 = create_ngram(text1, text2, n=q, pad=pad)
    
    full_text = list(text1.union(text2))
    v1 = [(lambda x: 1 if x in text1 else 0)(x) for x in full_text]
    v2 = [(lambda x: 1 if x in text2 else 0)(x) for x in full_text]    
    return spatial.distance.cosine(v1, v2)

In [ ]:
def ngram(text, n=3, pad=True):
    text = text.strip()
    if pad:
        text = " %s " % text
    return set([text[i:i+n] for i in range(len(text)-n+1)])
    
def create_ngram(text1, text2, n=3, pad=True):
    return ngram(text1, n=n, pad=pad), ngram(text2, n=n, pad=pad)

In [ ]:
def qgrams(text1, text2, q=3, pad=True):
    text1, text2 = create_ngram(text1, text2, n=q, pad=pad)
    return len(text1.intersection(text2)),text1.intersection(text2)

len,lis=qgrams(contents,d1['2173 Software engineers and designers'], q=10, pad=False) # 3

In [ ]:
def jaccard_dist(text1, text2, q=3, pad=True):
    text1, text2 = create_ngram(text1, text2, n=q, pad=pad)
    
    full_text = list(text1.union(text2))
    agree_tot = len(text1.intersection(text2))
    return 1 - agree_tot/float(len(full_text))

jaccard_dist(contents,d1['2173 Software engineers and designers'], q=2, pad=False) #0.5

In [ ]:
def tversky_index(text1, text2, a=None, b=None, q=3, pad=True):
    text1, text2 = create_ngram(text1, text2, n=q, pad=pad)
    agree_tot = len(text1.intersection(text2))
    v1 = len(text1) - agree_tot
    v2 = len(text2) - agree_tot
    
    if a != None and b != None:
        a = a/float(a+b)
        b = b/float(a+b)
    elif a <= 1.0 and a >= 0.0:
        b = 1-a
    elif b <= 1.0 and b >= 0.0:
        a = 1-b
    else:
        a = 0.5
        b = 0.5        
    return float(agree_tot)/(agree_tot+a*v1+b*v2)
    
tversky_index(contents,d1['2173 Software engineers and designers'], a=0.5, q=2, pad=False) # 0.6666666

In [ ]:
text

In [ ]:
cos_dist(contents,d1['2173 Software engineers and designers'] , q=2, pad=False)

In [ ]:
d1['2173 Software engineers and designers']